Create dataset png from single stl obj

In [2]:
import numpy as np
from stl import mesh
from mpl_toolkits import mplot3d
from matplotlib import pyplot as plt
import cv2
import os

def generate_dataset(mesh_file, output_dir, angles, image_size=(512, 512)):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    your_mesh = mesh.Mesh.from_file(mesh_file)

    for i, angle in enumerate(angles):
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')

        ax.add_collection3d(mplot3d.art3d.Poly3DCollection(your_mesh.vectors))

        scale = your_mesh.points.flatten()
        ax.auto_scale_xyz(scale, scale, scale)

        ax.view_init(elev=angle[0], azim=angle[1])

        ax.axis('off')

        image_path = os.path.join(output_dir, f'image_{i}.png')
        plt.savefig(image_path, bbox_inches='tight', pad_inches=0)
        plt.close(fig)

        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        image_resized = cv2.resize(image, image_size)

        _, segmented = cv2.threshold(image_resized, 1, 255, cv2.THRESH_BINARY)


        mask_path = os.path.join(output_dir, f'mask_{i}.png')
        cv2.imwrite(mask_path, segmented)
        cv2.imwrite(image_path, image_resized)

angles = [(90, 90), (90, 0), (0, 90), (0, 0), (45, 45)]

mesh_file = '../data/moon.stl'
output_dir = '../output'

generate_dataset(mesh_file, output_dir, angles)


Recovery 3d obj from image

In [ ]:
import torch
import requests
import numpy as np
from io import BytesIO
from diffusers import DiffusionPipeline
from PIL import Image

pipeline = DiffusionPipeline.from_pretrained(
    "dylanebert/LGM-full",
    custom_pipeline="dylanebert/LGM-full",
    torch_dtype=torch.float16,
    trust_remote_code=True,
).to("cuda")

input_url = "../output/image_2.png"
input_image = Image.open(BytesIO(requests.get(input_url).content))
input_image = np.array(input_image, dtype=np.float32) / 255.0
result = pipeline("", input_image)
result_path = "../output/recovery_2.ply"
pipeline.save_ply(result, result_path)

WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.4.0+cu121 with CUDA 1201 (you have 2.4.0+cpu)
    Python  3.9.13 (you have 3.9.13)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
D:\Program\pyth\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 'Could not find module 'D:\Program\pyth\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


Fetching 20 files:   0%|          | 0/20 [00:00<?, ?it/s]

(…)ature_extractor/preprocessor_config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

lgm/config.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

image_encoder/config.json:   0%|          | 0.00/563 [00:00<?, ?B/s]

...

unet/mv_unet.py:   0%|          | 0.00/34.4k [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

Keyword arguments {'trust_remote_code': True} are not expected by LGMFullPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/8 [00:00<?, ?it/s]

ImportError: This modeling file requires the following packages that were not found in your environment: plyfile, diff_gaussian_rasterization. Run `pip install plyfile diff_gaussian_rasterization`